In [9]:
import tensorflow as tf
import cv2
import numpy as np
import os

from keras.src.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential, load_model

# **Loading Data**

In [31]:
path_data = "dataset_farsi/"
file = os.listdir(path_data)

# x, y
data, label = [], []

# Resize x, y number
x_size, y_size = 32, 32

for folder in file:
    folder_path = os.path.join(path_data, folder)

    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        image = cv2.imread(img_path)

        image = cv2.resize(image, (x_size, y_size))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


        data.append(image)
        label.append(int(folder))

data = np.array(data)
data = np.float64(data) / 255.0
data = np.expand_dims(data, axis=3)
label = np.array(label)

# **Train and Test**

In [34]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42, shuffle=True)

# **Define the model**

In [40]:
model = Sequential([
    Conv2D(50, (3, 3), activation='relu', input_shape=(x_size, y_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(16, activation='relu'),
    Dense(10)
])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 30, 30, 50)     │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 15, 15, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 13, 13, 16)     │         7,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 6, 6, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 10)             │           170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,854 (179.12 KB)

 Trainable params: 45,854 (179.12 KB)

 Non-trainable params: 0 (0.00 B)

# **Training the model**

In [41]:
cb = [ModelCheckpoint("Best_accuracy_model.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode="max")], EarlyStopping(monitor='val_loss', patience=8)

history = model.fit(x_train, y_train, batch_size=256, epochs=10, validation_data=(x_test, y_test), callbacks=cb)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7212 - loss: 0.8694
Epoch 1: val_accuracy improved from None to 0.97431, saving model to Best_accuracy_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 71ms/step - accuracy: 0.8808 - loss: 0.3824 - val_accuracy: 0.9743 - val_loss: 0.0859
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.9761 - loss: 0.0792
Epoch 2: val_accuracy improved from 0.97431 to 0.98150, saving model to Best_accuracy_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 84ms/step - accuracy: 0.9782 - loss: 0.0717 - val_accuracy: 0.9815 - val_loss: 0.0600
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9839 - loss: 0.0527
Epoch 3: val_accuracy improved from 0.98150 to 0.98494, saving model to Best_accuracy_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 82ms/step - accuracy: 0.9845 - loss: 0.0499 - val_accuracy: 0.9849 - val_loss: 0.0485
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.9857 - loss: 0.0453
Epoc

# **Testing the model**

In [43]:
out = model.predict(x_test)
model.evaluate(x_test, y_test)

500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9915 - loss: 0.0298


[0.029789412394165993, 0.9915000200271606]

In [79]:
print(f"Some predicted of test: index 0: {np.argmax(out[0])} index 5: {np.argmax(out[5])} index 100: {np.argmax(out[100])}")
print(f"Actual number of test images: index 0: {y_test[0]} index 5: {y_test[5]} index 100: {y_test[100]}")

Some predicted of test: index 0: 5 index 5: 1 index 100: 8
Actual number of test images: index 0: 5 index 5: 1 index 100: 8
